In [1]:
# Database Connection
import sqlalchemy
from sqlalchemy import create_engine, inspect, text

# Data Science
import pandas as pd
import numpy as np

# Data Vizualization
import seaborn as sns
import matplotlib.pyplot as plt

# to enable visulization
from sqlalchemy import create_engine, text


In [2]:
# Assign SQL variables
SQL_USERNAME = "postgres"
SQL_PASSWORD = "Dasani595!"
SQL_IP = "localhost"
PORT = 5432
DATABASE = "project_two_etl"

In [3]:
# Set up Engine
connection_string = f"postgresql+psycopg2://{SQL_USERNAME}:{SQL_PASSWORD}@{SQL_IP}:{PORT}/{DATABASE}"
engine = create_engine(connection_string)

In [4]:
# Create the inspector and connect it to the engine
inspector = inspect(engine)

# Collect the names of tables within the database
tables = inspector.get_table_names()

# Use the inspector to print the column names within each table and its data types
for table in tables:
    print(table)
    columns = inspector.get_columns(table)
    for column in columns:
        print(column["name"], column["type"])
        
    print()

contacts
contact_id INTEGER
first_name VARCHAR(100)
last_name VARCHAR(250)
email VARCHAR(250)
last_updated TIMESTAMP

campaign
cf_id INTEGER
contact_id INTEGER
company_name VARCHAR(100)
description VARCHAR(500)
goal INTEGER
pledged INTEGER
outcome VARCHAR(100)
backers_count INTEGER
country VARCHAR(5)
currency VARCHAR(5)
launched_date DATE
end_date DATE
category_id VARCHAR(5)
subcategory_id VARCHAR(100)
last_updated TIMESTAMP

category
category_id VARCHAR(5)
category VARCHAR(100)
last_updated TIMESTAMP

subcategory
subcategory_id VARCHAR(8)
subcategory VARCHAR(100)
last_updated TIMESTAMP



In [5]:
# Retrieve full contacts table from Postgres
query = """
    SELECT *
    FROM contacts
    """
contacts = pd.read_sql(text(query), con=engine)
contacts.head()

,contact_id,first_name,last_name,email,last_updated
0,4661,Cecilia,Velasco,cecilia.velasco@rodrigues.fr,2024-07-16 20:14:01.661561
1,3765,Mariana,Ellis,mariana.ellis@rossi.org,2024-07-16 20:14:01.661561
2,4187,Sofie,Woods,sofie.woods@riviere.com,2024-07-16 20:14:01.661561
3,4941,Jeanette,Iannotti,jeanette.iannotti@yahoo.com,2024-07-16 20:14:01.661561
4,2199,Samuel,Sorgatz,samuel.sorgatz@gmail.com,2024-07-16 20:14:01.661561


In [6]:
# Query for 'US'
query = """
    SELECT
        cat.category,
        cam.outcome,
        COUNT(cam.outcome) AS count,
        cam.country,
        cam.launched_date,
        cam.end_date
        
    FROM campaign AS cam
    JOIN category AS cat ON (cam.category_id = cat.category_id)
    WHERE cam.outcome IN ('successful', 'failed')
        AND cam.country = 'US'
    GROUP BY cat.category, cam.outcome, cam.country, cam.launched_date, cam.end_date
    ORDER BY cat.category, cam.outcome DESC;
    """
question_3us = pd.read_sql(text(query), con=engine)
question_3us.head()

,category,outcome,count,country,launched_date,end_date
0,film & video,successful,6,US,2020-02-01,2021-02-08
1,film & video,successful,6,US,2020-03-03,2021-03-10
2,film & video,successful,6,US,2020-03-21,2021-05-13
3,film & video,successful,6,US,2020-03-28,2021-03-08
4,film & video,successful,6,US,2020-04-12,2021-03-24


In [7]:
question_3us['campaign_length'] = (question_3us['end_date'] - question_3us['launched_date'])
question_3us.dtypes

category           object
outcome            object
count               int64
country            object
launched_date      object
end_date           object
campaign_length    object
dtype: object

In [8]:
# Retrieve full category table from Postgres
query = """
    SELECT *
    FROM category
    """
category = pd.read_sql(text(query), con=engine)
category.head()

,category_id,category,last_updated
0,cat1,food,2024-07-16 20:14:01.733375
1,cat2,music,2024-07-16 20:14:01.733375
2,cat3,technology,2024-07-16 20:14:01.733375
3,cat4,theater,2024-07-16 20:14:01.733375
4,cat5,film & video,2024-07-16 20:14:01.733375


In [9]:
# Retrieve full subcategory table from Postgres
query = """
    SELECT *
    FROM subcategory
    """
subcategory = pd.read_sql(text(query), con=engine)
subcategory.head()

,subcategory_id,subcategory,last_updated
0,subcat1,food trucks,2024-07-16 20:14:01.750587
1,subcat2,rock,2024-07-16 20:14:01.750587
2,subcat3,web,2024-07-16 20:14:01.750587
3,subcat4,plays,2024-07-16 20:14:01.750587
4,subcat5,documentary,2024-07-16 20:14:01.750587


In [10]:
# Retrieve full campaign table from Postgres
query = """
    SELECT *
    FROM campaign
    """

campaign = pd.read_sql(text(query), con=engine)
campaign.head()

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,category_id,subcategory_id,last_updated
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,2020-02-13,2021-03-01,cat1,subcat1,2024-07-16 21:30:47.441202
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,2021-01-25,2021-05-25,cat2,subcat2,2024-07-16 21:30:47.441202
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,2020-12-17,2021-12-30,cat3,subcat3,2024-07-16 21:30:47.441202
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,2021-10-21,2022-01-17,cat2,subcat2,2024-07-16 21:30:47.441202
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,2020-12-21,2021-08-23,cat4,subcat4,2024-07-16 21:30:47.441202


In [11]:
# Calculate the length of the campaign in days
campaign['campaign_length'] = (campaign['end_date'] - campaign['launched_date'])
campaign

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,category_id,subcategory_id,last_updated,campaign_length
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,2020-02-13,2021-03-01,cat1,subcat1,2024-07-16 21:30:47.441202,"382 days, 0:00:00"
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,2021-01-25,2021-05-25,cat2,subcat2,2024-07-16 21:30:47.441202,"120 days, 0:00:00"
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,2020-12-17,2021-12-30,cat3,subcat3,2024-07-16 21:30:47.441202,"378 days, 0:00:00"
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,2021-10-21,2022-01-17,cat2,subcat2,2024-07-16 21:30:47.441202,"88 days, 0:00:00"
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,2020-12-21,2021-08-23,cat4,subcat4,2024-07-16 21:30:47.441202,"245 days, 0:00:00"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,2986,3684,Manning-Hamilton,Vision-oriented scalable definition,97300,153216,successful,2043,US,USD,2020-12-29,2021-05-30,cat1,subcat1,2024-07-19 22:12:38.949321,"152 days, 0:00:00"
5996,2031,5784,Butler LLC,Future-proofed upward-trending migration,6600,4814,failed,112,US,USD,2021-10-15,2021-11-30,cat4,subcat4,2024-07-19 22:12:38.949321,"46 days, 0:00:00"
5997,1627,1498,Ball LLC,Right-sized full-range throughput,7600,4603,canceled,139,IT,EUR,2021-11-06,2021-12-10,cat4,subcat4,2024-07-19 22:12:38.949321,"34 days, 0:00:00"
5998,2175,6073,"Taylor, Santiago and Flores",Polarized composite customer loyalty,66600,37823,failed,374,US,USD,2020-10-08,2021-04-11,cat2,subcat8,2024-07-19 22:12:38.949321,"185 days, 0:00:00"


In [12]:
import re
end_date = []
for row in campaign['campaign_length']:
    x = re.findall(r'\d+', row)
    x_int = int(x)
    end_date.append(x_int)
end_date

TypeError: expected string or bytes-like object

In [ ]:
from datetime import timedelta

duration = timedelta(days=int(campaign.campaign_length.split()[0]))
days = duration.days
print(days)

In [ ]:


# # Drop non-numeric values in the campaign_length column
# campaign['campaign_length'] = pd.to_numeric(campaign['campaign_length'], errors='coerce')
# campaign.dropna(subset=['campaign_length'], inplace=True)

# # Display the DataFrame after dropping non-numeric values
campaign.loc[campaign['outcome'] == 'canceled']


In [ ]:
# # Dataset
# campaign

# # Plot for question1us
# sns.barplot(campaign, x='outcome', y='campaign_length', hue='outcome', palette='prism')

# # customizations
# plt.xlabel("Outcome", fontstyle="italic", fontsize=12)
# plt.ylabel("Length of Campaign", fontstyle="italic", fontsize=12)
# plt.title("Campaign length vs Success or failure", fontsize=16, fontweight="bold")
# plt.xticks(rotation=90)

# plt.grid(color="lightgrey", axis="y", linestyle="--", alpha=0.75)

# # save chart in Charts folder
# plt.savefig('../Charts/campaign_length_vs_success_or_failure_png', bbox_inches='tight')

# # show
# plt.show()


In [ ]:
# Dataset
campaign

# Plot for campaign
sns.barplot(data=campaign, x='outcome', y='campaign_length', hue='outcome', palette='prism')

# Customizations
plt.xlabel("Outcome", fontstyle="italic", fontsize=12)
plt.ylabel("Length of Campaign", fontstyle="italic", fontsize=12)
plt.title("Campaign length vs Success or failure", fontsize=16, fontweight="bold")
plt.xticks(rotation=90)

plt.grid(color="lightgrey", axis="y", linestyle="--", alpha=0.75)

# Save chart in Charts folder
plt.savefig('../Charts/campaign_length_vs_success_or_failure.png', bbox_inches='tight')

# Show the plot
plt.show()


In [ ]:
# Close Connection
engine.dispose()